<a href="https://colab.research.google.com/github/min207/2023-1-ESAA/blob/main/ESAA230324.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# import package
import numpy as np
import os

## **Ch.07 앙상블 학습과 랜덤 포레스트**

- **앙상블 학습**: 일련의 예측기(분류나 회귀모델)로부터 예측을 수집하면 가장 좋은 모델 하나보다 더 좋은 예측을 얻는 것이 가능
  + 이때, 일련의 예측기를 앙상블, 앙상블 학습 알고리즘을 앙상블 방법이라고 함 
  + 훈련 세트로부터 무작위로 각기 다른 서브셋을 만들어 일련의 결정트리 분류기를 훈련, 모들 개별 트리의 예측을 구한 후 가장 많은 선택을 받은 클래스를 예측으로 결정

- **랜덤 포레스트**: 결정트리의 앙상블
- 그 외에도 배깅, 부스팅, 스태킹 등의 앙상블 방법이 존재

### 7.1 투표 기반 분류기

- **직접 투표** 분류기(hard voting): 각 분류기의 예측을 모아서 가장 많이 선택된 클래스를 예측하는 것

- **약한 학습기**(weak learner): 랜덤 추측보다 조금 더 높은 성능을 내는 분류기
  + 약한 학습기어도 충분하게 많고 다양하면 강한 학습기(strong learner)가 될 수 있음
  + 큰 수의 법칙에 의해 랜덤 추측보다 조금 더 나은 분류기를 여러개 사용해 앙상블 모델을 만들 수록 정확도가 더 올라감
    + 이는 모든 분류기가 완벽하게 독립적이고 오차에 상관관계가 없어야 가능 

In [1]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
X,y = make_moons(n_samples=100, noise=0.15)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators = [('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)], voting='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [3]:
# 정확도 확인
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train, y_train)
  y_pred=clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.7
RandomForestClassifier 1.0
SVC 0.9
VotingClassifier 0.9


> - 투표 기반 분류기가 다른 개별 분류기보다 성능이 조금 더 높음 


- **간접 투표**(soft voting): 모든 분류기가 클래스의 확률을 예측할 수 있으면(predict_proba()메서드), 개별 분류기의 예측을 평균 내어 확률이 가장 높은 클래스를 예측할 수 있음
  + 이 방식을 사용하기 위해서는 voting 파라미터를 "soft"로 지정
  + SVC는 기본값에서 클래스 확률을 제공하지 않기 때문에 probability 매개변수를 True로 지정정

### 7.2 배깅과 페이스팅
- **배깅**(bagging): 훈련 세트의 서브셋을 무작위로 구성하여 분류기를 각기 다르게 학습 시킬 때, 훈련 세트에서 중복을 허용하여 샘플링하는 방식
- **페이스팅**(pasting): 배깅과 반대로, 훈련 세트에서 중복을 허용하지 않고 샘플링하는 방식
  + 즉, 두 방법 모두 같은 훈련 샘플을 여러 개의 예측기에 걸쳐 사용할 수 있지만, 배깅만 한 예측기를 위해 같은 훈련 샘플을 여러 번 샘플링 할 수 있음


- 모든 예측기가 훈련을 마치면 앙상블은 모든 예측기의 예측을 모아서 새로운 샘플에 대한 예측을 생성
  + 수집 함수는 보편적으로 분류일때는 통계적 최빈값, 회귀일때는 평균을 계산
  + 수집 함수를 통과하면 편향과 분산 모두 감소
- 예측기는 모두 동시에 다른 CPU 코어나 서버에서 병렬로 학습시키는 것이 가능(예측도 병렬 수행 가능): 확장성이 높음 

#### 7.2.1 사이킷런의 배깅과 페이스팅
- 사이킷런은 배깅과 페이스팅을 위해 간편한 API로 구성된 BaggingClassifier(BaggingRegressor)을 제공
  + 위 메서드를 이용해 결정 트리 분류기 500개의 앙상블 훈련(배깅 사용)
  + 페이스팅을 사용하려면 bootstrap=False로 지정정

In [4]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500, max_samples=50, bootstrap=True, n_jobs=-1)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

- 단일 결정 트리의 결정 경계와 배깅 앙상블의 결정 결계를 비교하면, 앙상블의 경우가 비슷한 편향에서 더 작은 분산을 만들어 일반화가 더 잘 된 것을 확인 가능 

- 부트스트래핑은 각 예측기가 학습하는 서브셋에 다양성을 증가시키기 때문에 배깅이 페이스팅보다 편향이 조금 더 높음 
  + 그러나 다양성을 추가한다는 것은 예측기들의 상관관계를 감소시키기 때문에 앙상블의 분산이 감소 
    + 따라서 배깅이 전반적으로 더 나은 모델을 만듦 

#### 7.2.2 oob 평가

- 배깅을 사용할 경우 어떤 샘플은 여러 번 샘플링되고 어떤 샘플은 전혀 선택되지 않을 수 있음(보통 훈련 샘플의 63%만 샘플링)
  + **oob 샘플**: 배깅 과정에서 한 번도 선택되지 않은 훈련 샘플의 37% 

- 따라서 oob 샘플을 활용하여 별도의 검증 세트 없이 예측기 평가 가능 
- 사이킷런의 BaggingClassifier에서 obb_score=True로 지정하면 자동으로 oob 평가 수행 

In [14]:
# oob 평가
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, oob_score=True, random_state=40)
bag_clf.fit(X_train, y_train)

bag_clf.oob_score_

0.9125

In [15]:
# 실제 정확도
from sklearn.metrics import accuracy_score

y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

1.0

In [16]:
bag_clf.oob_decision_function_

array([[0.        , 1.        ],
       [0.23834197, 0.76165803],
       [0.04494382, 0.95505618],
       [0.8988764 , 0.1011236 ],
       [0.83152174, 0.16847826],
       [0.97674419, 0.02325581],
       [0.69662921, 0.30337079],
       [0.        , 1.        ],
       [0.67647059, 0.32352941],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.96464646, 0.03535354],
       [0.        , 1.        ],
       [0.88953488, 0.11046512],
       [0.93604651, 0.06395349],
       [0.18536585, 0.81463415],
       [0.07614213, 0.92385787],
       [0.95121951, 0.04878049],
       [0.95854922, 0.04145078],
       [0.31818182, 0.68181818],
       [0.72631579, 0.27368421],
       [0.76331361, 0.23668639],
       [0.55958549, 0.44041451],
       [0.85106383, 0.14893617],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [1.

### 7.3 랜덤 패치와 랜덤 서브스페이스

- BaggingClassifier는 특성 샘플링도 지원: max_features, bootstrap_features 두 매개변수로 조절
  + 작동 방식은 max_samples, bootstrap과 동일하지만 샘플이 아니고 특성에 대한 샘플링
  + 고차워늬 데이터셋을 다룰 때 유용하게 사용됨

- **랜덤 패치 방식**(random patches method): 훈련 특성과 샘플을 모두 샘플링하는 것
- **랜덤 서브스페이스 방식**(random subspaces method): 훈련 샘플을 모두 사용하고 특성만 샘플링하는 것
  + 특성 샘플링은 더 다양한 예측기를 만들며 편향을 늘리는 대신 분산을 감소시킴 

### 7.4 랜덤 포레스트
- 랜덤 포레스트는 일반적으로 배깅 방법을 적용한 결정 트리의 앙상블
  + 전형적으로 max_samples를 훈련 세트의 크기로 지정
  + RandomForestClassifier 사용 

In [11]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

>- RandomForestClassifier는 몇 가지 예외가 있지만 DecisionTreeClassifier의 매개변수와 앙상블 자체를 제어하는 데 필요한 BaggingClassifier의 매개변수를 모두 가지고 있음 
  + 트리의 노드를 분할할 때 전체 특성 중에서 최선의 특성을 찾는 대신 무작위로 선택한 특성 후보 중에서 최적의 특성을 찾는 식으로 무작위성을 주입
    + 편향을 손해보는 대신 분산을 낮추어 (다양성 추가) 더 훌륭한 모델을 생성 

In [12]:
# BaggingClassifier을 RandomforestClassifier와 비슷하게 구현
bag_clf=BaggingClassifier(DecisionTreeClassifier(max_features='auto', max_leaf_nodes=16),
                          n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1)

#### 7.4.1 엑스트라 트리
- 랜덤 포레스트에서 트리를 만들 때, 각 노드는 무작위로 특성의 서브셋을 만들어 분할에 사용
  +**익스트림 랜덤 트리**(엑스트라 트리): 극단적으로 트리를 더 무작위하게 만들기 위해 최적의 임곗값을 찾는 대신 후보 특성을 사용해 무작위로 분할 후 그중에서 최상의 분할 선택
    + 일반적인 랜덤 포레스트보다 속도가 훨씬 빠름 
    + ExtraTreeClassifier 사용 

#### 7.4.2 특성 중요도
- 랜덤 포레스트는 특성의 상대적 중요도를 측정하기 쉽다는 장점이 있음
  + 사이킷런은 어떤 특성을 사용한 노드가 평균적으로 불순도를 얼마나 감소시키는지 확인하여 특성의 중요도 측정(가중치 평균)
  + feature_importances_ 변수에 저장(중요도의 전체 합은 1)

In [13]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris['data'], iris['target'])
for name, score in zip(iris['feature_names'], rnd_clf.feature_importances_):
  print(name, score)

sepal length (cm) 0.10777068436298591
sepal width (cm) 0.026290369090814277
petal length (cm) 0.43154337110969054
petal width (cm) 0.43439557543650925


> - 특성을 선택해야할 때 중요도를 빠르게 파악할 수 있어 편리함함

### 7.5 부스팅
- **부스팅**: 약한 학습기를 여러 개 연결하여 강한 학습기를 만드는 앙상블 방법
  + ex) 에이다부스트(adaptive boosting), 그레이디언트 부스팅(gradient boosting)

#### 7.5.1 에이다부스트
- 이전 예측기를 보완하기 위해 이전 모델이 과소적합했던 훈련 샘플의 가중치를 더 높이는 방식 사용
  + 학습하기 어려운 샘플에 새로운 예측기가 더 맞춰짐

- 첫 번째 분류기가 많은 샘플을 잘못 분류하면 그 샘플들의 가중치를 높임
  + 두 번째 분류기는 그 샘플들을 더 정확히 예측

- 경사 하강법이 비용 함수를 최소화하기 위해 한 예측기의 모델 파라미터를 조정해가는 것처럼 에이다부스트는 앙상블에 가중치가 업데이트 된 예측기를 추가


- 모든 예측기가 훈련을 마치면 이 앙상블은 배깅이나 페이스팅과 비슷한 방식으로 예측 생성
  + 가중치가 적용된 훈련 세트의 전반넉인 정확도에 따라 예측기마다 다른 가중치 적용

-------------------------
- 각 샘플의 가중치 $w^{(i)}$을  $\frac{1}{m}$으로 초기화
- 첫 번째 예측기 학습 후 가중치가 적용된 에러율 $r_1$ 계산
$$r_j = \frac{\sum_{i=1, \hat{y_j} \ne y^{(i)}}^{m}w^{(i)}}{\sum_{i=1}^{m} w^{(i)}}$$
- 예측기의 가중치 $\alpha_j$ 계산(예측기가 정확할 수록 가중치는 높아짐)
$$\alpha_j= \eta log{\frac{1-r_j}{r_j}}$$
- 이후 에이다부스트 알고리즘이 샘플의 가중치 업데이트(잘못 분류된 샘플의 가중치 증가)
  $$ w^{(i)} = 
  \begin{cases}
  w^{(i)}, & \mbox{if }\hat{y_j} = y^{(i)} \\
  w^{(i)} exp(\alpha_j), & \mbox{if }\hat{y_j} \ne y^{(i)}
  \end{cases} $$
- 마지막으로 모든 샘플의 가중치 정규화
---------------------------
- 위의 과정을 반복하여 지정된 예측기 수에 도달하거나 완벽한 예측기가 되면 중지 

- 에이다부스트는 단순히 모든 예측기의 예측을 계산하고 예측기 가중치 $\alpha_j$를 더해 예측 결과 생성
  + 가중치 합이 가장 큰 클래스가 예측 결과
- 사이킷런은 SAMME라는 에이다부스트의 다중 클래스 버전 사용
  + 클래스가 두 개인 경우 SAMME와 에이다부스트 동일
  + 예측기가 클래스의 확률을 추정할 수 있는 경우 SAMME.R이라는 SAMME의 변종 사용용

In [17]:
# AdaBoostClassifier의 기본 추정기
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5, random_state=42)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200, random_state=42)

#### 7.5.2 그레이디언트 부스팅
- 앙상블에 이전까지의 오차를 보정하도록 예측기를 순차적으로 추가하는 방식
  + 에이다부스트와 다르게 샘플의 가중치를 수정하는 대신 이전 예측기가 만든 **잔여 오차**에 새로운 예측기를 학습습

In [19]:
# 데이터 저장
np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3*X[:, 0]**2 + 0.05 * np.random.randn(100)

In [21]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2, random_state=1)
tree_reg1.fit(X, y)

DecisionTreeRegressor(max_depth=2, random_state=1)

> - 첫 번째 예측기에서 생긴 잔여 오차에 두 번째 DecisionTreeRegressor

In [22]:
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2, random_state=1)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(max_depth=2, random_state=1)

> - 두 번째 예측기가 만든 잔여 오차에 세 번째 회귀 모델 훈련 

In [25]:
y3 = y - tree_reg2.predict(X) 
tree_reg3 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(max_depth=2, random_state=42)

> - 세 개의 트리를 포함하는 앙상블 모델
  + 새로운 샘플에 대한 예측을 만들기 위해 모든 트리의 예측을 더함 

In [23]:
# 테스트셋 생성
X_new = np.array([[0.8]])

In [26]:
y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

> - 왼쪽 열은 이 세 트리의 예측이고 오른쪽 열 앙상블의 예측
  - 첫 번째 행에서는 앙상블에 트리가 하나만 있어서 첫 번째 트리의 예측과 완전히 동일
  - 두 번째 행에서는 새로운 트리가 첫 번째 트리의 잔여 오차에 대해 학습
    + 오른쪽의 앙상블 예측이 두 개의 트리 예측의 합과 동일
  - 세 번째 행에서는 또 다른 트리가 두 번째 트리의 잔여 오차에 훈련

- 트리가 앙상블에 추가될수록 앙상블의 예측이 점차 좋아짐 



--------------
- 사이킷런의 GradientBoostingRegressor를 사용하면 GBRT 앙상블을 간단하게 훈련시킬 수 있음 

In [27]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0, random_state=1)
gbrt.fit(X, y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3,
                          random_state=1)

> - learning_rate 매개변수가 각 트리의 기여 정도를 조절 
    + **축소**: 이를 0.1처럼 낮게 설정하면 앙상블을 훈련 세트에 학습시키기 위해 많은 트리가 필요하지만 일반적으로 예측의 성능이 좋아지는 규제 방법 

In [31]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=1)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120, random_state=1)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred)
          for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors) + 1

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators, random_state=1)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=78, random_state=1)

> - warm_start=True로 설정하면 사이킷런이 fit() 메서드가 호출될 때 기존 트리를 유지하고 훈련을 추가할 수 있음 
  - 다섯 번의 반복 동안 검증 오차가 향상되지 않으면 훈련을 멈춤 

In [33]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True, random_state=1) 

min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error 
        error_going_up = 0
    else:
        error_going_up += 1 
        if error_going_up == 5:
            break  # 조기 종료 

> - GradientBoostingRegressor는 각 트리가 훈련할 때 사용할 훈련 샘플의 비율을 지정할 수 있는 subsample 매개변수 지원 
- subsample=0.25라고 하면 각 트리는 무작위로 선택된 25%의 훈련 샘플로 학습
- 편향이 높아지는 대신 분산이 낮아짐 : 이러한 기법을 **확률적 그레이디언트 부스팅**이라고 함



- 최적화된 그레이디언트 부스팅 구현으로 XGBoost 파이썬 라이브러리가 유명 

In [34]:
import xgboost

xgb_reg = xgboost.XGBRegressor(random_state=1)
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

In [35]:
# 자동 조기 종료 기능
xgb_reg.fit(X_train, y_train, 
            eval_set=[(X_val, y_val)], early_stopping_rounds=2)
y_pred = xgb_reg.predict(X_val)

[0]	validation_0-rmse:0.24376
[1]	validation_0-rmse:0.17987
[2]	validation_0-rmse:0.13545
[3]	validation_0-rmse:0.10585
[4]	validation_0-rmse:0.08634
[5]	validation_0-rmse:0.07435
[6]	validation_0-rmse:0.06727
[7]	validation_0-rmse:0.06336
[8]	validation_0-rmse:0.06293
[9]	validation_0-rmse:0.06148
[10]	validation_0-rmse:0.06131
[11]	validation_0-rmse:0.06104
[12]	validation_0-rmse:0.06105
[13]	validation_0-rmse:0.06161


/usr/local/lib/python3.9/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


### 7.6 스태킹

- **스태킹**: 앙상블에 속한 모든 예측기의 예측을 취합하는 간단한 함수를 사용하는 대신 취합하는 모델을 훈련시키는 아이디어로 출발하는 방법 
  + 마지막 예측기를 **블렌더** 또는 **메타 학습기**라고 함

- 블렌더를 학습시키는 일반적인 방법은 홀드 아웃 세트를 사용하는 것 
  + 훈련 세트를 두 개의 서브셋으로 나눔
  + 첫 번째 서브셋은 첫 번째 레이어의 예측을 훈련시키기 위해 사용
  + 첫 번째 레이어의 예측기를 사용해 두 번째 세트에 대한 예측 생성
  + 홀드 아웃 세트의 각 샘플에 대해 세 개의 예측값이 있음 
  + 타깃값은 그대로 사용하고 앞에서 예측한 값을 입력 특성으로 사용하는 새로운 훈련 세트 생성
  + 블렌더가 새 훈련 세트로 훈련 